### `NOTE`: Run this notebook on a high memory node

In [ ]:
import os
from joblib import Parallel, delayed
from itertools import repeat

from pathlib import Path
from importlib import reload


import numpy as np
import pandas as pd
from astropy.table import Table, vstack, hstack, join, unique
import fitsio


from desitarget.targets import encode_targetid
from desimodel.footprint import radec2pix
from desitarget.targets import desi_mask, scnd_mask
from desitarget.sv1 import sv1_targetmask
from desitarget.sv2 import sv2_targetmask
from desitarget.sv3 import sv3_targetmask
from desispec.zcatalog import find_primary_spectra

import matplotlib.pyplot as plt
from tqdm import tqdm

### Read the redshift Catalog

In [ ]:
release = "iron" #loa
version = "1"

zcatalog = Path(f"/global/cfs/cdirs/desi/spectro/redux/{release}/zcatalog/v{version}/zall-pix-{release}.fits")

dr = 1
version = "1.1"
photo_cat = Path(f"/global/cfs/cdirs/desi/vac/dr{dr}/lsdr9-photometry/{release}/v{version}/observed-targets")

version = "3.0"
fastspec_cat = Path(f"/global/cfs/cdirs/desi/public/dr{dr}/vac/dr{dr}/fastspecfit/{release}/v{version}/catalogs")

dest_path = Path(os.environ["SCRATCH"]) / "data" / "foundation" / f"{release}"

In [ ]:
zcatalog = Table(fitsio.read(zcatalog))

In [ ]:
zQuality = (zcatalog["Z"]>0.001)
zQuality &= zcatalog["ZWARN"]==0
zQuality &= zcatalog["OBJTYPE"]=="TGT"
zQuality &= zcatalog["SPECTYPE"]!="STAR"
zQuality &= zcatalog["COADD_FIBERSTATUS"]==0
zQuality &= zcatalog["DELTACHI2"]>30
zQuality &= zcatalog["ZCAT_PRIMARY"]
zQuality &= zcatalog["SURVEY"] != "cmx"

In [ ]:
zcatalog = zcatalog[zQuality]
zcatalog.remove_column("PHOTSYS") #there is a bug so using PHOTSYS from photocat

In [ ]:
len(zcatalog)

### Match with the photometry VAC and the fastspec VAC for QSO redshifts

In [ ]:
phot_cols = []
for file in tqdm(photo_cat.glob("*.fits")):
    phot_cols.append(Table(fitsio.read(file,columns=['TARGETID','NOBS_G','NOBS_R','NOBS_Z',"PHOTSYS"])))

In [ ]:
phot_cols = vstack(phot_cols)
phot_cols = unique(phot_cols,keys="TARGETID")

In [ ]:
fastspec_cols = []
for file in tqdm(fastspec_cat.glob("*.fits")):
    fastspec_cols.append(Table(fitsio.read(file, ext=1, columns=["TARGETID", "Z" , "ZWARN", "TSNR2_LRG"])))

In [ ]:
fastspec_cols = vstack(fastspec_cols)
nspec, spec_primary = find_primary_spectra(fastspec_cols)
fastspec_cols = fastspec_cols[spec_primary]

In [ ]:
zcatalog.rename_column('Z', 'Z_RR')
fastspec_cols.remove_columns(["ZWARN", "TSNR2_LRG"])

zcatalog = join(zcatalog,fastspec_cols,keys="TARGETID",join_type="left")
zcatalog = join(zcatalog,phot_cols,keys="TARGETID",join_type="left")

In [ ]:
len(zcatalog)

In [ ]:
# photoCut = (zcatalog["NOBS_G"]>0)
# photoCut |= (zcatalog["NOBS_R"]>0)
# photoCut |= (zcatalog["NOBS_Z"]>0)
photoCut = (zcatalog["NOBS_R"]>0)

In [ ]:
zcatalog = zcatalog[photoCut]

In [ ]:
def fluxToMag(flux):
    return 22.5 - 2.5 * np.log10(flux.clip(1e-16))

def mw_transmission(cat,band):
    band_coeffs = {"u":3.995, "g": 3.214, "r": 2.165, "i": 1.592, "z": 1.211, "Y": 1.064,
              "W1": 0.184,"W2": 0.113,"W3": 0.0241,"W4": 0.00910}
    A = band_coeffs[band]*cat["EBV"]
    return 10**(A/-2.5)

In [ ]:
zcatalog["MW_TRANSMISSION_G"] = mw_transmission(zcatalog,"g")
zcatalog["MW_TRANSMISSION_R"] =  mw_transmission(zcatalog,"r")
zcatalog["MW_TRANSMISSION_Z"] = mw_transmission(zcatalog,"z")
zcatalog["MW_TRANSMISSION_W1"] = mw_transmission(zcatalog,"W1")
zcatalog["MW_TRANSMISSION_W2"] = mw_transmission(zcatalog,"W2")


zcatalog["gmag"] = fluxToMag(zcatalog["FLUX_G"] / zcatalog["MW_TRANSMISSION_G"])
zcatalog["rmag"] = fluxToMag(zcatalog["FLUX_R"] / zcatalog["MW_TRANSMISSION_R"])
zcatalog["zmag"] = fluxToMag(zcatalog["FLUX_Z"] / zcatalog["MW_TRANSMISSION_Z"])
zcatalog["w1mag"] = fluxToMag(zcatalog["FLUX_W1"] / zcatalog["MW_TRANSMISSION_W1"])
zcatalog["w2mag"] = fluxToMag(zcatalog["FLUX_W2"] / zcatalog["MW_TRANSMISSION_W2"])




### Put in final selection cuts

In [ ]:
mask_test = (zcatalog["rmag"]<19.5) & (zcatalog["rmag"]>12) #& (zcatalog["Z"]<1.6)
print(mask_test.sum())

#BGS faint is 20.175 in r
rmask = np.isfinite(zcatalog["rmag"])
plt.hist(zcatalog["rmag"][rmask&mask_test],bins=100,histtype="step",label="r-mag")
plt.xlabel("mag")
plt.ylabel("COUNT")
plt.yscale("log")
plt.legend()


In [ ]:
plt.hist(zcatalog["Z"][mask_test],bins=100)
plt.xlabel("DESI Spectroscopic redshift")
plt.ylabel("COUNT")
plt.yscale("log")

In [ ]:
len(zcatalog)

In [ ]:
mag_cut = (zcatalog["rmag"]<19.5) & (zcatalog["rmag"]>12)
zcatalog = zcatalog[mag_cut]
zcatalog["MYID"] = np.arange(len(zcatalog))
zcatalog.remove_column('COEFF')




In [ ]:
len(zcatalog)

In [ ]:
zcatalog.to_pandas().to_parquet(dest_path / "desi_zcat_maglim_19_5.parquet")

### Row matched fastspec fit

In [ ]:
hdu1 = []
hdu2 = []
hdu3 = []

for file in tqdm(fastspec_cat.glob("*.fits")):
    hdu1.append(Table(fitsio.read(file, ext=1,)))
    hdu2.append(Table(fitsio.read(file, ext=2, )))
    hdu3.append(Table(fitsio.read(file, ext=3, )))
    
hdu1 = vstack(hdu1)
hdu1.remove_column("TILEID_LIST")
hdu2 = vstack(hdu2)
hdu2.remove_columns(["SURVEY", "PROGRAM", "HEALPIX", "COEFF"])
hdu3 = vstack(hdu3)
hdu3.remove_columns(["SURVEY", "PROGRAM", "HEALPIX",])

                
                
hdu1 = join(hdu1,hdu2,keys="TARGETID",join_type="left")
hdu1 = join(hdu1,hdu3,keys="TARGETID",join_type="left")

In [ ]:
nspec, spec_primary = find_primary_spectra(hdu1)
hdu1 = hdu1[spec_primary]

In [ ]:
hdu1 = join(zcatalog[["MYID", "TARGETID"]],hdu1,keys="TARGETID",join_type="left")

In [ ]:
hdu1.to_pandas().to_parquet(dest_path / "desi_fastspec_maglim_19_5.parquet")

In [ ]:
len(hdu1)